In [2]:
import pandas as pd
import json

In [13]:
js = pd.read_json('tmdb.json')
df = pd.DataFrame(js)
df

,model,pk,fields
0,movies.genre,28,{'name': '액션'}
1,movies.genre,12,{'name': '모험'}
2,movies.genre,16,{'name': '애니메이션'}
3,movies.genre,35,{'name': '코미디'}
4,movies.genre,80,{'name': '범죄'}
...,...,...,...
10005,movies.movie,505225,"{'adult': False, 'genre_ids': [18, 53], 'origi..."
10006,movies.movie,10988,"{'adult': False, 'genre_ids': [9648, 53, 27], ..."
10007,movies.movie,1051,"{'adult': False, 'genre_ids': [28, 80, 53], 'o..."
10008,movies.movie,871875,"{'adult': False, 'genre_ids': [27, 9648, 10751..."


In [50]:
df['fields'][100]['title']

'토르: 러브 앤 썬더'

In [53]:
#genre_ids=[]
#popularity=[]
title=[]

for i in range(len(df)):
    #genre_ids.append(df['fields'][i][["genre_ids"]])
    #popularity.append(df['fields'][i]["popularity"])
    title.append(df['fields'][i]['title'])
        
df2 = pd.DataFrame({
    "title":title,
})

print(df2)


KeyError: 'title'

In [1]:
from tmdbv3api import TMDb, Movie

tmdb = TMDb()
tmdb.api_key = "234c446afa7c9156e6c9a101b724f5ae"

movie = Movie()

m = movie.details(111)

print(m.title)
print(m.overview)
print(m.popularity)


Scarface
After getting a green card in exchange for assassinating a Cuban government official, Tony Montana stakes a claim on the drug trade in Miami. Viciously murdering anyone who stands in his way, Tony eventually becomes the biggest drug lord in the state, controlling nearly all the cocaine that comes through Miami. But increased pressure from the police, wars with Colombian drug cartels and his own drug-fueled paranoia serve to fuel the flames of his eventual downfall.
83.822


In [6]:
import requests


def SearchMovies(query, year):  # 영화 제목과 년도 정보를 받음

    params = {"api_key": "234c446afa7c9156e6c9a101b724f5ae",
          "language": "ko",
          "page": "1",
          "query": query,
          "include_adult": "true",
          "year": year}


    url = "https://api.themoviedb.org/3/search/movie" # 정보를 요청할 주소
    resp = requests.get(url, params=params)
    data = resp.json()['results']


    if(len(data) > 0): # 검색해서 나온 데이터가 있을 경우에만

       for item in data : # 반복문 돌면서 영화 이름과 제작년도 출력
           print(item['original_title'], item['release_date']) 

if __name__ == '__main__':
    SearchMovies('존 윅',"2023")

John Wick: Chapter 4 2023-03-22
